# Zarr Tiling 

Notebook for taking Opera Phenix tiff output and tiling the images together into zarr mosaics with metadata attribute, specifically for the heterognosis project

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import zarr
import dask.array
from scipy import stats
import xml.etree.ElementTree as ET
from tqdm.auto import tqdm 
from skimage.io import imread
from macrohet import dataio 


def create_alpha_mask(height, width, feather=0.1):
    """
    Create a 2D alpha mask of shape (height, width).
    The mask is 1 in the interior, and over the last `feather` pixels
    near each edge, it linearly ramps down to 0.
    feather is a fraction of image i.e. a float percentage.
    This is the core function that enables smooth image tiling 
    without dramatic cropping of cell morphology.
    """
    # Coordinates
    y = np.arange(height)
    x = np.arange(width)

    feather_pixels = feather * min(height, width)
    
    # Distance from left and right edges
    dist_x_left  = x
    dist_x_right = (width - 1) - x
    # Distance from top and bottom edges
    dist_y_top    = y
    dist_y_bottom = (height - 1) - y
    
    # For each row, figure out the "vertical" ramp factor from top/bottom
    # We'll make an array that is the min distance to top or bottom.
    ramp_y = np.minimum(dist_y_top, dist_y_bottom)
    # For each column, min distance to left or right
    ramp_x = np.minimum(dist_x_left, dist_x_right)
    
    # Combine to get a 2D "distance from any edge"
    # We can broadcast ramp_y over columns, ramp_x over rows
    dist_from_edge = np.minimum(
        ramp_y[:, None],
        ramp_x[None, :]
    )
    
    # Turn that distance into an alpha that goes from 0 to 1 linearly
    # from the very edge to `feather` pixels in.
    # If dist_from_edge >= feather, alpha=1. If dist < 0, alpha=0, etc.
    alpha = np.clip(dist_from_edge / feather_pixels, 0, 1)
    
    return alpha


def load_assay_layout(xml_file_name):
    """
    Loads and parses an assay layout XML file into a pandas DataFrame with a 'position' index.

    Args:
        xml_file_name (str): The full path to the assay layout XML file.

    Returns:
        pandas.DataFrame: A DataFrame containing the assay layout information,
                          with a 'position' column as index and each layer as a column.
                          Returns an empty DataFrame if the file is not found or parsing fails.
    """
    df_assay_layout = pd.DataFrame()

    if not os.path.exists(xml_file_name):
        print(f"Error: The file '{xml_file_name}' was not found.")
    else:
        try:
            tree = ET.parse(xml_file_name)
            root = tree.getroot()
            namespaces = {'ns': 'http://www.perkinelmer.com/PEHH/HarmonyV5'}
            plate_data = {}

            for layer in root.findall('ns:Layer', namespaces):
                layer_name_element = layer.find('ns:Name', namespaces)
                if layer_name_element is None or layer_name_element.text is None:
                    print("Warning: Found a layer without a name. Skipping this layer.")
                    continue
                layer_name = layer_name_element.text.strip()

                for well in layer.findall('ns:Well', namespaces):
                    row_element = well.find('ns:Row', namespaces)
                    col_element = well.find('ns:Col', namespaces)
                    value_element = well.find('ns:Value', namespaces)

                    if row_element is None or row_element.text is None or \
                       col_element is None or col_element.text is None:
                        print(f"Warning: Found a well in layer '{layer_name}' without row or col. Skipping this well.")
                        continue

                    try:
                        row = int(row_element.text.strip())
                        col = int(col_element.text.strip())
                    except ValueError:
                        print(f"Warning: Could not parse row/col as integers for a well in layer '{layer_name}'. Skipping this well.")
                        continue

                    value = value_element.text.strip() if value_element is not None and value_element.text is not None else pd.NA
                    well_key = (row, col)
                    if well_key not in plate_data:
                        plate_data[well_key] = {}
                    plate_data[well_key][layer_name] = value

            if plate_data:
                df_assay_layout = pd.DataFrame.from_dict(plate_data, orient='index')
                df_assay_layout.index.names = ['Row', 'Col']
                df_assay_layout = df_assay_layout.sort_index()
                df_assay_layout = df_assay_layout.dropna(how='all')

                # Create a new 'position' column
                df_assay_layout['position'] = df_assay_layout.index.map(lambda rc: f"({rc[0]}, {rc[1]})")
                # Set 'position' as the new index
                df_assay_layout = df_assay_layout.set_index('position')
                df_assay_layout = df_assay_layout.drop(columns=['Row', 'Col']) # Drop the old index columns

            else:
                print("No data was extracted from the XML file to populate the DataFrame.")

        except ET.ParseError as e:
            print(f"Error parsing XML file '{xml_file_name}': {e}")
        except FileNotFoundError:
            print(f"Error: The file '{xml_file_name}' was not found.")
        except Exception as e:
            print(f"An unexpected error occurred during XML processing: {e}")

    return df_assay_layout


### Add acquisition subdirectories

This is where all the imagery will be kept

In [ ]:
base_dir = '/mnt/DATA3/BPP0050/'
parent_directories = glob.glob(os.path.join(base_dir, '*/'))

# Define the name of the new subdirectory for acquisitions
ACQUISITIONS_DIR_NAME = "acquisition"

print(f"Starting to organize subdirectories into '{ACQUISITIONS_DIR_NAME}'...")

for parent_dir in parent_directories:
    # Ensure the parent directory path is clean and exists
    parent_dir = os.path.normpath(parent_dir)
    if not os.path.isdir(parent_dir):
        print(f"Warning: Parent directory not found, skipping: {parent_dir}")
        continue

    # Construct the path for the new 'acquisitions' subdirectory
    acquisitions_path = os.path.join(parent_dir, ACQUISITIONS_DIR_NAME)

    # Create the 'acquisitions' subdirectory if it doesn't exist
    try:
        os.makedirs(acquisitions_path, exist_ok=True)
        print(f"Ensured '{acquisitions_path}' exists.")
    except OSError as e:
        print(f"Error creating '{acquisitions_path}': {e}")
        continue # Skip to the next parent directory if creation fails

    # List all entries in the parent directory
    # Filter for actual directories, excluding the 'acquisitions' directory itself
    items_in_parent_dir = [
        d for d in os.listdir(parent_dir)
        if os.path.isdir(os.path.join(parent_dir, d)) and d != ACQUISITIONS_DIR_NAME
    ]

    if not items_in_parent_dir:
        print(f"No subdirectories found to move in: {parent_dir}")
        continue

    print(f"Processing parent directory: {parent_dir}")
    for subdir_name in items_in_parent_dir:
        source_path = os.path.join(parent_dir, subdir_name)
        destination_path = os.path.join(acquisitions_path, subdir_name)

        try:
            # Move the subdirectory
            shutil.move(source_path, destination_path)
            print(f"  Moved '{source_path}' to '{destination_path}'")
        except shutil.Error as e:
            print(f"  Error moving '{source_path}': {e}")
        except Exception as e:
            print(f"  An unexpected error occurred while moving '{source_path}': {e}")

print("Script finished.")

# Tile, add metadata and save out 

In [3]:
base_dir = '/mnt/DATA3/BPP0050/'
base_dirs = glob.glob(os.path.join(base_dir, '*/'))
base_dirs

['/mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Fixed_Cy5__2025-04-18T16_38_51-Measurement 1/',
 '/mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Fixed_Cy8__2025-04-21T16_49_44-Measurement 1/',
 '/mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Fixed_Cy10__2025-04-23T15_45_30-Measurement 1/',
 '/mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Fixed_Cy2__2025-04-15T16_42_33-Measurement 3/',
 '/mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Fixed_Cy4__2025-04-17T16_24_52-Measurement 1/',
 '/mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Live-1__2025-04-09T18_25_04-Measurement 1/',
 '/mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Live-2__2025-04-10T18_45_48-Measurement 1/',
 '/mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Fixed_Cy7__2025-04-20T16_03_17-Measurement 1/',
 '/mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Fixed_Cy17__2025-04-30T18_09_02-Measurement 1/',
 '/mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Fixed_Cy13__2025-04-26T16_33_47-Measurement 1/',
 '/mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Fixed_Cy6__2025-0

In [8]:
base_dir.split('/')[-2]

'BPP0050-1-Live-cell-to4i_Live-2__2025-04-10T18_45_48-Measurement 1'

In [5]:
base_dir_ID

''

In [9]:


for base_dir in tqdm(base_dirs, total = len(base_dirs)):

    base_dir_ID = base_dir.split('/')[-2] #os.path.basename(base_dir)
    if base_dir_ID == 'BPP0050-1-Live-cell-to4i_Live-2__2025-04-10T18_45_48-Measurement 1':
        print('skipping untransferred', base_dir_ID)
        continue
    
    metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
    metadata = dataio.read_harmony_metadata(metadata_fn)  

    # Convert applicable columns to numeric values
    metadata = metadata.apply(pd.to_numeric, errors='ignore')
    # Invert Y coordinates (cathode ray tube adjustment)
    metadata['PositionY'] = -metadata['PositionY']
    
    # Find the minimum Z position for non-Phase Contrast channels
    min_z = metadata.loc[~metadata['ChannelName'].str.contains('Phase Contrast', case=False, na=False), 'PositionZ'].min()
    
    # Assign this minimum Z value to all "Phase Contrast" channels
    metadata.loc[metadata['ChannelName'].str.contains('Phase Contrast', case=False, na=False), 'PositionZ'] = min_z
    
    image_dir = os.path.join(base_dir, 'acquisition/Images/')
    # use the metadata for the rows cols as the assay layout can be the same for multiple acquisitions, encompassing multiple layouts
    rows_cols = metadata[['Row', 'Col']].drop_duplicates().values
    
    assay_layout_fn = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
    assay_layout = load_assay_layout(assay_layout_fn)




    for row_col in tqdm(rows_cols, desc=f'Now tiling in {base_dir_ID}', total = len(rows_cols)):
        row, column = int(row_col[0]), int(row_col[1])
        col = column
        acq_ID = row, column
        
        if os.path.exists(os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')):
            print(os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr'), 'already exists, skipping')
            continue

        # Filter metadata for the current row and column (include all time points)
        position_metadata = metadata[(metadata['Row'] == row) & (metadata['Col'] == column)]
        # Get unique time points
        timepoint_ids = sorted(position_metadata['TimepointID'].unique())
        # Initialize a list to store time-lapse volumes
        time_volumes = []
        
        sample_tile_id =  position_metadata.iloc[0]
        # Iterate over each time point
        for timepoint_id in tqdm(timepoint_ids, desc="Processing Time Points", leave=True, position=1):
            # Filter metadata for the current time point
            time_metadata = position_metadata[position_metadata['TimepointID'] == timepoint_id]
            # Get unique Channel
            channel_ids = sorted(time_metadata['ChannelID'].unique())
            # Initialize a list to hold each channel's Z-stack for this time point
            channel_volumes = []
            # Iterate over each channel
            for channel_id in tqdm(channel_ids, desc=f"Processing Channels (T={timepoint_id})", leave=False, position=2):
                # get specific channel metadata
                channel_slice_metadata = time_metadata[time_metadata['ChannelID'] == channel_id]
                # get z positions for this channel
                z_ids = sorted(channel_slice_metadata['PositionZ'].unique())
                # init a list to hold z slices
                z_slices = []
                # Iterate over Z positions
                for z_id in z_ids:#tqdm(z_positions, desc="Processing Z-slices", leave=False, position=3):
                    # get specific z slice metadata
                    z_slice_metadata = channel_slice_metadata[channel_slice_metadata['PositionZ'] == z_id]
                    # Initialize mosaic_slice if needed, ie for the first image to be placed in the blank slate
                    mosaic_size_x = int((z_slice_metadata['PositionX'].max() - z_slice_metadata['PositionX'].min()) / sample_tile_id['ImageResolutionX']) + sample_tile_id['ImageSizeX']
                    mosaic_size_y = int((z_slice_metadata['PositionY'].max() - z_slice_metadata['PositionY'].min()) / sample_tile_id['ImageResolutionY']) + sample_tile_id['ImageSizeY']
                    # For each position, create a large mosaic (accumulator & weight)
                    # Suppose mosaic_size_y, mosaic_size_x is the total bounding box in Y and X.
                    accumulator = np.zeros((mosaic_size_y, mosaic_size_x), dtype=np.float32)
                    weight_map  = np.zeros((mosaic_size_y, mosaic_size_x), dtype=np.float32)
                    
                    for tile_index, tile_id in z_slice_metadata.iterrows():
                        # load the tile
                        img_path = os.path.join(image_dir, tile_id['URL'])
                        try:
                            tile = imread(img_path).astype(np.float32)
                        except FileNotFoundError:
                            tile = np.zeros((tile_id['ImageSizeY'], tile_id['ImageSizeX']), dtype=np.float32)
                    
                        # create alpha mask for this tile's shape
                        alpha_mask = create_alpha_mask(tile.shape[0], tile.shape[1], feather=0.1)
                    
                        # figure out where to place this tile in the mosaic
                        x_pixel = int((tile_id['PositionX'] - z_slice_metadata['PositionX'].min()) / tile_id['ImageResolutionX'])
                        y_pixel = int((tile_id['PositionY'] - z_slice_metadata['PositionY'].min()) / tile_id['ImageResolutionY'])
                    
                        # add into the accumulator & weight
                        # We slice out the region in the mosaic that corresponds to this tile
                        acc_slice = accumulator[y_pixel:y_pixel+tile.shape[0], x_pixel:x_pixel+tile.shape[1]]
                        wgt_slice = weight_map[y_pixel:y_pixel+tile.shape[0], x_pixel:x_pixel+tile.shape[1]]
                    
                        acc_slice += tile * alpha_mask
                        wgt_slice += alpha_mask
                    
                    # AFTER all tiles are placed:
                    final_mosaic = np.zeros_like(accumulator, dtype=np.float32)
                    
                    # Avoid division by zero
                    nonzero_mask = (weight_map > 0)
                    final_mosaic[nonzero_mask] = accumulator[nonzero_mask] / weight_map[nonzero_mask]
                    
                    # Then, convert final mosaic to e.g. uint16 if desired
                    mosaic = np.clip(final_mosaic, 0, 65535).astype(np.uint16)
                    
                    # now append that mosaic z slice to z slices stack, different if DPC
                    if "Phase Contrast" in channel_slice_metadata['ChannelName'].iloc[0]:  
                        # print(f"Distributing DPC image across Z slices")
                        dpc_slice = mosaic / len(metadata['PositionZ'].unique())  
                        z_slices = [mosaic] * len(metadata['PositionZ'].unique())  
                        break  
                    else:
                        z_slices.append(mosaic)
                    
                #stack z slices into a channel volume
                if len(z_slices) > 0:
                    # channel_volume = np.max(z_slices, axis=0)
                    channel_volume = np.stack(z_slices, axis=0)
                    channel_volumes.append(channel_volume)
            # stack the channel volume into a time volume
            if len(channel_volumes) > 0:
                image_volume_czyx = np.stack(channel_volumes, axis=0)  # C, Z, Y, X
                time_volumes.append(image_volume_czyx)  # Store for this time point
        # Stack all time points into a 5D array (T, C, Z, Y, X)
        if len(time_volumes) > 0:
            images = np.stack(time_volumes, axis=0)  # T, C, Z, Y, X
            print(f"Final image volume shape: {images.shape}")

        try:
            zarr_output_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
            dask_images = dask.array.from_array(images)
            zarr_group = zarr.open(zarr_output_dir, mode='w')
            dask.array.to_zarr(dask_images, zarr_output_dir, component='images')
            print(f"Saved Zarr array successfully to {zarr_output_dir}")
        except:
            print(base_dir_ID, acq_ID, 'zarr output failed')

    # --- Omero Metadata ---
    omero_metadata = {}
    average_time_difference_seconds = None

    # Calculate temporal framerate based on unique, sorted TimepointIDs
    if 'AbsTime' in metadata.columns and 'TimepointID' in metadata.columns:
        try:
            if not pd.api.types.is_datetime64_any_dtype(metadata['AbsTime']):
                metadata['AbsTime'] = pd.to_datetime(metadata['AbsTime'], format='ISO8601', utc=True)

            timepoint_abs_times = metadata[['TimepointID', 'AbsTime']].drop_duplicates(subset=['TimepointID']).sort_values(key=lambda x: pd.to_numeric(x, errors='ignore'), by='TimepointID')

            if len(timepoint_abs_times['TimepointID'].unique()) > 1: # Check if there is a time series
                # Calculate the time difference between consecutive timepoints in seconds
                time_diffs = timepoint_abs_times['AbsTime'].diff().dt.total_seconds().dropna()

                if not time_diffs.empty:
                    # Calculate the average time difference in seconds
                    average_time_difference_seconds = time_diffs.mean()
                    if average_time_difference_seconds > 0:
                        framerate = 1 / average_time_difference_seconds
                        omero_metadata['frameRate'] = framerate
                        print(f"Calculated framerate: {framerate} frames per second")
                    else:
                        print("Warning: Average time difference is zero or negative, cannot calculate framerate.")
                else:
                    print("Warning: Could not calculate time differences for framerate.")
            else:
                print("Warning: Less than two unique TimepointIDs found, skipping framerate calculation.")

        except Exception as e:
            print(f"Warning: An error occurred while calculating framerate: {e}")
    # Channels (rest of your channel metadata code remains the same)
    channels_data = []
    for index, row in metadata.drop_duplicates(subset=['ChannelID']).iterrows():
        emission_wavelength_meters = None
        excitation_wavelength_meters = None
    
        if pd.notna(row['MainEmissionWavelength']):
            try:
                emission_wavelength = float(row['MainEmissionWavelength'])
                emission_wavelength_meters = emission_wavelength * 1e-9
            except ValueError:
                print(f"Warning: Could not convert '{row['MainEmissionWavelength']}' to float for emission wavelength (Channel ID: {row['ChannelID']}). Setting to None.")
    
        if pd.notna(row['MainExcitationWavelength']):
            try:
                excitation_wavelength = float(row['MainExcitationWavelength'])
                excitation_wavelength_meters = excitation_wavelength * 1e-9
            except ValueError:
                print(f"Warning: Could not convert '{row['MainExcitationWavelength']}' to float for excitation wavelength (Channel ID: {row['ChannelID']}). Setting to None.")
    
        channel = {
            "id": int(row['ChannelID']),
            "label": row['ChannelName'],
            "emissionWaveMeters": emission_wavelength_meters,
            "excitationWaveMeters": excitation_wavelength_meters,
            # Add other relevant channel metadata if available
        }
        channels_data.append(channel)
    omero_metadata['channels'] = channels_data
    
    # Objective (rest of your objective metadata code remains the same)
    objective_data = {}
    if 'ObjectiveMagnification' in metadata.columns and pd.notna(metadata['ObjectiveMagnification'].iloc[0]):
        try:
            objective_data['magnification'] = float(metadata['ObjectiveMagnification'].iloc[0])
        except ValueError:
            print(f"Warning: Could not convert '{metadata['ObjectiveMagnification'].iloc[0]}' to float for objective magnification. Skipping.")
    if 'ObjectiveNA' in metadata.columns and pd.notna(metadata['ObjectiveNA'].iloc[0]):
        try:
            objective_data['numericalAperture'] = float(metadata['ObjectiveNA'].iloc[0])
        except ValueError:
            print(f"Warning: Could not convert '{metadata['ObjectiveNA'].iloc[0]}' to float for objective NA. Skipping.")
    if objective_data:
        omero_metadata['objective'] = objective_data

    # --- Multiscales Metadata ---
    multiscales_data = [{
        "name": "images",
        "datasets": [],
        "axes": [
            {"name": "t", "type": "time", "unit": "second", "spacing": average_time_difference_seconds},
            {"name": "c", "type": "channel"},
            {"name": "z", "type": "space", "unit": "meter", "spacing": None},
            {"name": "y", "type": "space", "unit": "meter", "pixelResolution": None},
            {"name": "x", "type": "space", "unit": "meter", "pixelResolution": None}
        ],
        "type": "image",
        "version": "0.4"
    }]

    # Iterate through your existing Zarr directories to populate datasets (remains the same)
    zarr_root_dir = f'{base_dir}/acquisition/zarr/'
    for filename in os.listdir(zarr_root_dir):
        if filename.endswith('.zarr'):
            position_name = filename.replace('.zarr', '')
            multiscales_data[0]['datasets'].append({"path": f"{filename}/images"})

    # Add Image resolution and size information (remains the same)
    if 'ImageResolutionX' in metadata.columns and 'ImageResolutionY' in metadata.columns:
        try:
            resolution_x = float(metadata['ImageResolutionX'].iloc[0])
            resolution_y = float(metadata['ImageResolutionY'].iloc[0])
            multiscales_data[0]['axes'][3]['pixelResolution'] = resolution_y
            multiscales_data[0]['axes'][4]['pixelResolution'] = resolution_x
        except ValueError:
            print("Warning: Could not convert ImageResolutionX or ImageResolutionY to float. Skipping resolution metadata.")
        except IndexError:
            print("Warning: Could not set pixelResolution, check axes configuration.")

    if 'ImageSizeX' in metadata.columns and 'ImageSizeY' in metadata.columns:
        try:
            size_x = int(metadata['ImageSizeX'].iloc[0])
            size_y = int(metadata['ImageSizeY'].iloc[0])
        except ValueError:
            print("Warning: Could not convert ImageSizeX or ImageSizeY to int. Skipping image size metadata.")

    # Calculate mode of differences between unique Z positions for pseudo z-resolution (remains the same)
    if 'PositionZ' in metadata.columns:
        try:
            z_positions = metadata['PositionZ'].astype(float).unique()
            z_positions_sorted = np.sort(z_positions)
            if len(z_positions_sorted) > 1:
                z_diffs = np.diff(z_positions_sorted)
                mode_result = stats.mode(z_diffs)
                if mode_result.count > 0:
                    modal_z_diff = mode_result.mode
                    multiscales_data[0]['axes'][2]['spacing'] = modal_z_diff # in meters
                    multiscales_data[0]['axes'][2]['unit'] = "meter"
                else:
                    print("Warning: No unique mode found for z-position differences.")
            else:
                print("Warning: Only one unique z position found, cannot calculate z difference mode.")
        except ValueError:
            print("Warning: Could not process PositionZ values for z-resolution calculation.")
        except IndexError:
            print("Warning: Could not set z spacing, check axes configuration.")


    # --- Combine Metadata and save ---
    top_level_attrs = {
        "assay_layout": assay_layout.to_dict(orient='index'),
        "omero": omero_metadata,
        "multiscales": multiscales_data,
        "version": "0.4",
    }

    # Create the top-level Zarr group and save metadata
    group = zarr.group(zarr_root_dir, overwrite=False)
    group.attrs.put(top_level_attrs)

    print(f"OME-Zarr metadata and assay layout saved to: {zarr_root_dir}/.zattrs")

  0%|          | 0/20 [00:00<?, ?it/s]

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!


/tmp/ipykernel_8701/2682895781.py:12: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  metadata = metadata.apply(pd.to_numeric, errors='ignore')


...

Final image volume shape: (1, 4, 8, 6911, 6911)
Saved Zarr array successfully to /mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Fixed_Cy14__2025-04-27T15_20_15-Measurement 1/acquisition/zarr/(5, 4).zarr


Processing Time Points:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Channels (T=0):   0%|          | 0/4 [00:00<?, ?it/s]

Final image volume shape: (1, 4, 8, 6911, 6911)
Saved Zarr array successfully to /mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Fixed_Cy14__2025-04-27T15_20_15-Measurement 1/acquisition/zarr/(5, 5).zarr
OME-Zarr metadata and assay layout saved to: /mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Fixed_Cy14__2025-04-27T15_20_15-Measurement 1//acquisition/zarr//.zattrs


/tmp/ipykernel_8701/2682895781.py:152: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  timepoint_abs_times = metadata[['TimepointID', 'AbsTime']].drop_duplicates(subset=['TimepointID']).sort_values(key=lambda x: pd.to_numeric(x, errors='ignore'), by='TimepointID')


# redo this one: '/mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Live-2__2025-04-10T18_45_48-Measurement 1/acquisition/Images/Index.idx.xml'
